## Implementation of models

This notebook can only be executed if there already is a folder containing all the low resolution images and a folder containing all the high resolution images.

##### Important: When doing a training on new data adjust the upscaling factor in the SRCNN model's bicubic interpolation!

### Imports

In [1]:
from PIL import Image
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import os
from torch.utils.data import DataLoader, Dataset
import math
from torchvision.transforms import ColorJitter, Normalize
from torch.utils.data import ConcatDataset
from torch.utils.data import Subset


### data loading (with data augmentation)

In [ ]:
low_res_folder = "./drive/MyDrive/buildings_low_res"
high_res_folder = "./drive/MyDrive/buildings"


# === creating dataset with all images ===
class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):
        self.low_res_images = sorted(os.listdir(low_res_folder))
        self.high_res_images = sorted(os.listdir(high_res_folder))
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, index):
        low_res_image = Image.open(os.path.join(low_res_folder, self.low_res_images[index]))
        high_res_image = Image.open(os.path.join(high_res_folder, self.high_res_images[index]))

        if self.transform is not None:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

# transform to tensor
base_transform = transforms.Compose([
    transforms.ToTensor()
])

# original dataset
dataset = CustomDataset(low_res_folder, high_res_folder, transform=base_transform)



# === Splitting into train and val sets ===

train_size = 0.8  # Proportion of data to be used for training
dataset_size = len(dataset)
split = int(train_size * dataset_size)
train_indices = list(range(split))
val_indices = list(range(split, dataset_size))

# Create train dataset as a subset of the combined dataset
train_dataset = Subset(dataset, train_indices)

# Create val dataset as a subset of the combined dataset
val_dataset = Subset(dataset, val_indices)



# === Normalization ===
normalize = Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

train_dataset = [(normalize(image), normalize(target)) for image, target in train_dataset]

val_dataset = [(normalize(image), normalize(target)) for image, target in val_dataset]



# === train_data augmentation ===

# color jitter augmentation for training
train_color_jitter = ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)

# augmentation factor
augmentation_factor = 5

# augmented datasets with random color jitter
augmented_datasets = []
for _ in range(augmentation_factor):
    augmented_dataset = []
    for image, target in train_dataset:
        augmented_dataset.append((train_color_jitter(image), train_color_jitter(target)))
    augmented_datasets.append(augmented_dataset)

# combine original and augmented datasets
combined_datasets = [train_dataset] + augmented_datasets
combined_dataset = ConcatDataset(combined_datasets)



# === final data loaders ===

# Data loaders for train and val sets
batch_size = 32
train_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Number of samples in each set
print(f"Number of training samples originally: {len(train_dataset)}, now augmented to: {len(combined_dataset)}")
print(f"Number of val samples: {len(val_dataset)}")


### FSRCNN implementation

In [ ]:
class FSRCNN(nn.Module):
    def __init__(self, d=56, s=12, m=4):
        super(FSRCNN, self).__init__()
        # Feature Extraction
        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.relu1 = nn.PReLU(d)
        # Shrinking
        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.relu2 = nn.PReLU(s)
        # Non-linear Mapping
        self.mapping = nn.Sequential(*[nn.Sequential(
            nn.Conv2d(s, s, kernel_size=3, padding=1),
            nn.PReLU(s)
        ) for _ in range(m)])
        # Expanding
        self.conv3 = nn.Conv2d(s, d, kernel_size=1)
        self.relu3 = nn.PReLU(d)
        # Deconvolution
        self.deconv = nn.ConvTranspose2d(d, 3, kernel_size=9, stride=3, padding=4, output_padding=1)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.mapping(x)
        x = self.relu3(self.conv3(x))
        x = self.deconv(x)
        return x

# Training hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# instance of the CNN model
model = FSRCNN().to(device)

# hyperparameters
learning_rate = 0.0001
num_epochs = 50

# loss function and optimizer
criterion = nn.MSELoss() # note: standard MSE is used, PSNR normally not used for training (just as metric at the end)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training process
for epoch in range(num_epochs):
    for input_data, desired_data in train_loader:
        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training loss per epoch
    psnr = 10 * math.log10(1 / loss.item())
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, PSNR: {psnr}")

# saving the model
torch.save(model.state_dict(), "FSRCNN.pth")

### SRCNN implementation

In [ ]:
# SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=3, mode='bicubic')
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.interpolation(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.relu3(self.conv3(x))
        return x

# Training hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# instance of the CNN model
model = SRCNN().to(device)

# hyperparameters
learning_rate = 0.001
num_epochs = 40

# loss function and optimizer
criterion = nn.MSELoss() # note: standard MSE is used, PSNR normally not used for training (just as metric at the end)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training process
for epoch in range(num_epochs):
    for input_data, desired_data in train_loader:
        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training loss per epoch
    psnr = 10 * math.log10(1 / loss.item())
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, PSNR: {psnr}")

# saving the model
torch.save(model.state_dict(), "SRCNN.pth")